### Project 3: Machine Translation
For this project, I want to make use of the Pivot approach and use a Pretrained English-Tagalog model to translate to Cebuano and Hiligaynon as these are the languages I speak.

In [1]:
from utils import *

In [2]:
from transformers import MarianMTModel, MarianTokenizer

# Load English → Tagalog model
en_tl_model_name = "Helsinki-NLP/opus-mt-en-tl"
en_tl_tokenizer = MarianTokenizer.from_pretrained(en_tl_model_name)
en_tl_model = MarianMTModel.from_pretrained(en_tl_model_name)

def translate_en_to_tl(text):
    inputs = en_tl_tokenizer(text, return_tensors="pt", truncation=True)
    translated = en_tl_model.generate(**inputs)
    return en_tl_tokenizer.decode(translated[0], skip_special_tokens=True)

print(translate_en_to_tl("In the beginning God created the heaven and the earth."))


c:\Users\Asanro Jay\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Nang pasimula ay nilalang ng Diyos ang langit at ang lupa.


#### Creating Parallel Corpora by Reusing Function from MCO1

In [3]:
csv_files = {
    "Hiligaynon": ["Bible/CSV/Hiligaynon_Exodus.csv","Bible/CSV/Hiligaynon_Genesis.csv","Bible/CSV/Hiligaynon_Leviticus.csv"],
    "Tagalog": ["Bible/CSV/Tagalog_Exodus.csv","Bible/CSV/Tagalog_Genesis.csv","Bible/CSV/Tagalog_Leviticus.csv"],
    "Cebuano": ["Bible/CSV/Cebuano_Exodus.csv","Bible/CSV/Cebuano_Genesis.csv","Bible/CSV/Cebuano_Leviticus.csv"],
}
pairs = [
    ("Tagalog", "Hiligaynon"),
    ("Tagalog", "Cebuano"),
]

create_parallel_corpora(csv_files, pairs, "Parallel_Corpora")

Saved: Parallel_Corpora\Tagalog_Hiligaynon_parallel.csv
Saved: Parallel_Corpora\Tagalog_Cebuano_parallel.csv
